In [10]:
import cv2
import numpy as np

# localMaxima 를 찾는 함수 
def findLocalMaxima(src):
    #구성요소를 사각형 모델로 만들어서 
    kernel = cv2.getStructuringElement(shape = cv2.MORPH_RECT, ksize=(11,11))
    #팽창을 적용하면 우리가 분할한 이진 영상 255영역이 커진다

    dilate = cv2.dilate(src, kernel)
    # src 랑 dilate랑 같니???
    #픽셀값 하나하나를 서로 확인한다. 
    #그래서 각 필셀에 Bool 값이 들어간다.
    localMax = (src == dilate)
    
    #일반적인 데이터에 대해서 erode를 하면 커널 안의 가장 
    #작은 값이 저장이 된다.
    erode = cv2.erode(src, kernel)
    #소스랑 erode랑 비교해서 소스가 큰 값이 남게 된다.
    #그래서 local minium만 지우게 된다.
    localMax2 = (src > erode)
    #그래서 두개를 and 연산을 하면 localMax만 남는다.
    localMax &= localMax2
    #로컬 맥스에 트루값이 들어가는 위치(X, Y)!
    points = np.argwhere(localMax == True)
    #행렬은 (Y,X)로 데이터가 저장되기 때문에 X , Y로 바꿔준다.
    points[:,[0,1]] = points[:,[1,0]] # switch x , y
    return points

In [11]:
src = cv2.imread("./data/CornerTest.jpg")
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.preCornerDetect(gray, ksize = 3)
ret, res2 = cv2.threshold(np.abs(res), 0.1 , 0 , cv2.THRESH_TOZERO)
corners = findLocalMaxima(res2)
print(len(corners))

dst = src.copy()
for x , y in corners:
    cv2.circle(dst , (x, y), 5 , (0,0,255),2)

cv2.imshow("src", src)
cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

8


In [15]:
#cornerMinEigenValsAndVecs

src = cv2.imread("./data/CornerTest.jpg")
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
#MinEigenVal은 코너 값을 바로 가져올 수 있다.
eigen = cv2.cornerMinEigenVal(gray, blockSize = 5)
print("eigen.shape = ", eigen.shape)

T = 0.2 #쓰레숄드를 0.2로 지정 argwhre로 Threshold 사용
corners = np.argwhere(eigen > T)
corners[:,[0,1]] = corners[:,[1,0]]
print("len(corners) =", len(corners))
dst = src.copy()
for x , y in corners:
    cv2.circle(dst, (x, y), 3 , (0,0,255), 2)
    
cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

eigen.shape =  (512, 512)
len(corners) = 8


In [22]:
#CornerHaarris

src = cv2.imread("./data/CornerTest.jpg")
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

res = cv2.cornerHarris(gray, blockSize = 5, ksize = 3, k = 0.01)
ret,res = cv2.threshold(np.abs(res), 0.02, 0, cv2.THRESH_TOZERO)
corners = findLocalMaxima(res)
print("corners=", corners)

dst = src.copy()

for x , y in corners:
    cv2.circle(dst, (x, y), 3, (0,0,255), 2)

cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

corners= [[109 127]
 [264 127]
 [267 167]
 [386 170]
 [109 268]
 [167 271]
 [170 374]
 [386 374]]


In [26]:
#fast -- 확실한 코너가 아니라면 효과가 별로 좋지 않다.

src = cv2.imread("./data/CornerTest.jpg")
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

fast = cv2.FastFeatureDetector_create(30)
#0으로 했다는 것은 maxSuppression을 쓰지 않는다.
fast.setNonmaxSuppression(0)
kp = fast.detect(src, None)
dst = src.copy()
#결과를 쉽게 그릴 수 있따.
cv2.drawKeypoints(src, kp, dst, (0,0,255))

cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [28]:
#Circles Grid

src = cv2.imread("./data/circleGrid.jpg")
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
patternSize = (6,4)
found, centers = cv2.findCirclesGrid(src, patternSize)
print("centers.shape=", centers.shape)

dst = src.copy()
cv2.drawChessboardCorners(dst,patternSize, centers, found)

cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

centers.shape= (24, 1, 2)
